<a href="https://colab.research.google.com/github/bobbyantonio/iecdt_environmental_data_analysis/blob/main/Preprocessing_climate_data_hackton_8decv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install darts

In [2]:
import pandas as pd
import xarray as xr
import seaborn as sns
# from tqdm import tqdm
from scipy.stats import pearsonr
import numpy as np
from sklearn.metrics import r2_score
import os
from darts.models import TCNModel
import pandas as pd
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.missing_values import fill_missing_values
from darts.utils.likelihood_models import GaussianLikelihood, QuantileRegression
from darts.dataprocessing.transformers import Scaler, StaticCovariatesTransformer
from darts.models import TFTModel, NBEATSModel
# Assuming 'series' is your TimeSeries object
from darts import concatenate
from darts.utils.callbacks import TFMProgressBar
import shutil
import torch
from pytorch_lightning.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from darts.models import NBEATSModel
from darts.models import AutoARIMA
from darts.models import XGBModel
import warnings
#import optuna
# Suppress warnings
warnings.filterwarnings("ignore")
# 1. Suppress PyTorch Lightning Hardware Messages
import logging
logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)
logging.getLogger("pytorch_lightning.utilities.rank_zero").setLevel(logging.WARNING)
logging.getLogger("pytorch_lightning.accelerators.cuda").setLevel(logging.WARNING)
logging.getLogger("darts").setLevel(logging.WARNING)
# =============================================================================
# 1. HELPER: Date Alignment
# =============================================================================
def to_month_end(date_series):
    """Aligns dates to the last day of the month."""
    return pd.to_datetime(date_series) + pd.offsets.MonthEnd(0)

In [3]:
def to_month_end(date_series):
    """Aligns dates to the last day of the month."""
    return pd.to_datetime(date_series) + pd.offsets.MonthEnd(0)

In [4]:
name="DOM" #This string selects the data of the region to study, DOM is dominican republic, GRA Graneda etc


In [5]:
#Load of epidemological data
df_epi= pd.read_csv("%s_epi_training.csv"%name)

df_epi['Date'] = to_month_end(df_epi['Month'])
df_epi = df_epi.drop(columns=['Month', 'Unnamed: 0'], errors='ignore')
if df_epi.duplicated(subset=['Date']).any():
    df_epi = df_epi.groupby(['Date']).mean(numeric_only=True).reset_index()
df_epi


,Country,Region,Cases,Spatial_Resolution,Date
0,DOMINICAN REPUBLIC,DOMINICAN REPUBLIC,21,Admin0,2014-12-31
1,DOMINICAN REPUBLIC,DOMINICAN REPUBLIC,273,Admin0,2015-01-31
2,DOMINICAN REPUBLIC,DOMINICAN REPUBLIC,342,Admin0,2015-02-28
3,DOMINICAN REPUBLIC,DOMINICAN REPUBLIC,432,Admin0,2015-03-31
4,DOMINICAN REPUBLIC,DOMINICAN REPUBLIC,394,Admin0,2015-04-30
...,...,...,...,...,...
92,DOMINICAN REPUBLIC,DOMINICAN REPUBLIC,1070,Admin0,2022-08-31
93,DOMINICAN REPUBLIC,DOMINICAN REPUBLIC,1828,Admin0,2022-09-30
94,DOMINICAN REPUBLIC,DOMINICAN REPUBLIC,1920,Admin0,2022-10-31
95,DOMINICAN REPUBLIC,DOMINICAN REPUBLIC,1541,Admin0,2022-11-30


In [6]:

#Loading part of climatic data (standardized precipitation index and standardized evapotranporation index) in a file
#IMP
for i in ["01","03","06","12"]:
  spi=xr.open_dataset("%s-spa%s.zs.nc"%(name,i))
  spi=spi.where(spi.time.dt.day==1,drop=True).to_dataframe()#the first day of the month contains information of the last 30 days in SPI 1 and 3
  #For spi and spie 6 it only has 1 data per month so this line is harmeless

  spie=xr.open_dataset("%s-spe%s.zs.nc"%(name,i))
  spie=spie.where(spie.time.dt.day==1,drop=True).to_dataframe()#the first day of the month contains information of the last 30 days in SPI 1 and 3
  if i=="01":
   inter_data=spi
   inter_data["spe%s" %i]=spie
  else:
       inter_data["spa%s" %i]=spi
       inter_data["spe%s" %i]=spie
inter_data.drop(index=inter_data.index[-1],inplace=True)
inter_data=inter_data.reset_index()
inter_data["time"]=inter_data["time"].dt.to_period("M")
inter_data

,time,region,spa01,spe01,spa03,spe03,spa06,spe06,spa12,spe12
0,2015-01,DOM,-0.517534,-0.614625,-0.240362,-0.304545,-0.349382,-0.371714,-1.297219,-1.150566
1,2015-02,DOM,1.082209,1.113880,-0.182529,-0.233308,-0.269710,-0.304991,-1.026904,-0.938466
2,2015-03,DOM,1.029790,1.073085,0.773866,0.734489,-0.230838,-0.295462,-1.016961,-0.913970
3,2015-04,DOM,-0.247234,-0.309503,0.775546,0.760518,-0.151664,-0.130728,-1.126334,-1.031561
4,2015-05,DOM,-2.520282,-2.028042,-1.072317,-0.983117,-1.057238,-1.101611,-1.391256,-1.239282
...,...,...,...,...,...,...,...,...,...,...
125,2025-06,DOM,-0.742095,-0.667191,0.013596,-0.003682,-0.421099,-0.440258,0.187191,0.130215
126,2025-07,DOM,-0.684034,-0.491420,-0.192809,-0.140606,-0.539482,-0.549485,-0.108972,-0.172499
127,2025-08,DOM,-0.715331,-0.661631,-1.000523,-0.949739,-0.833009,-0.869668,-0.278329,-0.350477
128,2025-09,DOM,-1.409213,-1.535912,-1.271335,-1.135710,-0.506093,-0.510955,0.056568,0.023770


In [7]:
# Load of observation data (reanalysis)
reanalysis=xr.open_dataset('%s-reanalysis_monthly.zs.nc'%name)#.sel(time=slice("2014-12-31","2022-12-31")).to_dataframe()

reanalysis=reanalysis.to_dataframe().reset_index()
reanalysis["time"]=pd.to_datetime(reanalysis["time"]).dt.to_period("M")
reanalysis



,time,region,t2m,rh,tp
0,2015-01,DOM,296.737845,0.800612,0.307693
1,2015-02,DOM,296.779563,0.795463,0.508437
2,2015-03,DOM,297.144032,0.765326,0.228332
3,2015-04,DOM,298.317135,0.752357,0.178149
4,2015-05,DOM,299.044725,0.750203,0.313936
...,...,...,...,...,...
125,2025-06,DOM,299.744513,0.793690,0.385766
126,2025-07,DOM,300.181694,0.789661,0.364312
127,2025-08,DOM,300.752432,0.788375,0.404145
128,2025-09,DOM,300.267422,0.815065,0.932228


In [8]:

#load of climatic data
reanalysis=xr.open_dataset('%s-reanalysis_monthly.zs.nc'%name)#.sel(time=slice("2014-12-31","2022-12-31")).to_dataframe()

reanalysis=reanalysis.to_dataframe().reset_index()
reanalysis["time"]=pd.to_datetime(reanalysis["time"]).dt.to_period("M")
reanalysis


#Load of forecast data and group it into one dataset , forecast data will be classified as variable_LX, being LX the forecast week
# For examplke T2m_L2 is the forecast of temperature for the 2nd week when the forecast starts in "time"
#Load of reanlysis
forecast=xr.open_dataset("%s-seasonal_forecast_monthly.zs.nc"%name)

def unroll_months(ds: xr.Dataset) -> pd.DataFrame:
    if "month" not in ds.dims:
        raise ValueError("Only use this on seasonal_forecast_monthly data")
    months = ds['month'].values
    orig_vars = list(ds.data_vars)
    # create new month-sliced variables var_L1 ... var_L6 (each will be dims ('time','region'))
    for var in ds.data_vars:
        for m in months:
            # cast m to int if it's numeric for nicer names; use str(m) if not
            name = f"{var}_L{m}"
            ds[name] = ds[var].sel(month=m)

    ds = ds.drop_vars(orig_vars)
    if 'month' in ds.dims:
        ds = ds.drop_dims('month')
    return ds.to_dataframe().reset_index()
forecast=unroll_months(forecast)
forecast=forecast.reset_index()
forecast["time"]=pd.to_datetime(forecast["time"]).dt.to_period("M")
forecast

clim_data=pd.merge(reanalysis,forecast)
clim_data.columns.values

array(['time', 'region', 't2m', 'rh', 'tp', 'index', 't2m_L1.0',
       't2m_L2.0', 't2m_L3.0', 't2m_L4.0', 't2m_L5.0', 't2m_L6.0',
       'rh_L1.0', 'rh_L2.0', 'rh_L3.0', 'rh_L4.0', 'rh_L5.0', 'rh_L6.0',
       'tp_L1.0', 'tp_L2.0', 'tp_L3.0', 'tp_L4.0', 'tp_L5.0', 'tp_L6.0',
       'mx2t24_L1.0', 'mx2t24_L2.0', 'mx2t24_L3.0', 'mx2t24_L4.0',
       'mx2t24_L5.0', 'mx2t24_L6.0', 'mn2t24_L1.0', 'mn2t24_L2.0',
       'mn2t24_L3.0', 'mn2t24_L4.0', 'mn2t24_L5.0', 'mn2t24_L6.0'],
      dtype=object)

In [9]:
#including el nino index
nino=xr.open_dataset("elninossta.nc").to_dataframe().reset_index()
nino["time"]=pd.to_datetime(nino["time"]).dt.to_period("M")
nino


,time,ssta
0,2015-01,0.477419
1,2015-02,0.750000
2,2015-03,0.438710
3,2015-04,0.836667
4,2015-05,0.841935
...,...,...
126,2025-07,-0.083871
127,2025-08,-0.338710
128,2025-09,-0.423333
129,2025-10,-0.509677


In [10]:


clim_data=pd.merge(reanalysis,forecast)
clim_data.columns.values

array(['time', 'region', 't2m', 'rh', 'tp', 'index', 't2m_L1.0',
       't2m_L2.0', 't2m_L3.0', 't2m_L4.0', 't2m_L5.0', 't2m_L6.0',
       'rh_L1.0', 'rh_L2.0', 'rh_L3.0', 'rh_L4.0', 'rh_L5.0', 'rh_L6.0',
       'tp_L1.0', 'tp_L2.0', 'tp_L3.0', 'tp_L4.0', 'tp_L5.0', 'tp_L6.0',
       'mx2t24_L1.0', 'mx2t24_L2.0', 'mx2t24_L3.0', 'mx2t24_L4.0',
       'mx2t24_L5.0', 'mx2t24_L6.0', 'mn2t24_L1.0', 'mn2t24_L2.0',
       'mn2t24_L3.0', 'mn2t24_L4.0', 'mn2t24_L5.0', 'mn2t24_L6.0'],
      dtype=object)

In [11]:
clim_data

,time,region,t2m,rh,tp,index,t2m_L1.0,t2m_L2.0,t2m_L3.0,t2m_L4.0,...,mx2t24_L3.0,mx2t24_L4.0,mx2t24_L5.0,mx2t24_L6.0,mn2t24_L1.0,mn2t24_L2.0,mn2t24_L3.0,mn2t24_L4.0,mn2t24_L5.0,mn2t24_L6.0
0,2015-01,DOM,296.737845,0.800612,0.307693,0,296.003048,295.954013,296.484124,297.253767,...,300.329790,300.992990,301.426667,302.432747,293.653464,293.378766,293.739177,294.505283,295.369165,296.234461
1,2015-02,DOM,296.779563,0.795463,0.508437,1,296.014714,296.464293,297.322312,298.178321,...,300.989378,301.630720,302.482374,303.229442,293.592826,293.830732,294.573811,295.542603,296.380792,296.735205
2,2015-03,DOM,297.144032,0.765326,0.228332,2,296.601700,297.458505,298.168917,299.055160,...,301.635922,302.500986,303.340244,303.619429,293.968395,294.677446,295.489687,296.385210,296.784302,296.974068
3,2015-04,DOM,298.317135,0.752357,0.178149,3,297.323178,298.121582,298.962766,299.674028,...,302.395492,303.424765,303.731316,303.526471,294.568748,295.494279,296.287564,296.788201,296.941981,296.986188
4,2015-05,DOM,299.044725,0.750203,0.313936,4,298.671638,299.405328,299.925757,300.036108,...,303.768514,303.935797,303.611964,302.807706,295.737503,296.548509,297.007120,297.173793,297.105166,296.762237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,2025-06,DOM,299.744513,0.793690,0.385766,125,299.156605,299.631656,299.853066,299.722307,...,303.600914,303.444969,302.536345,301.304584,296.373418,296.786608,297.098190,297.038888,296.614203,295.639563
126,2025-07,DOM,300.181694,0.789661,0.364312,126,299.479318,299.814824,299.657331,299.035313,...,303.347091,302.500054,301.353363,300.448711,296.639351,297.028190,296.991677,296.623259,295.686722,294.540026
127,2025-08,DOM,300.752432,0.788375,0.404145,127,299.626891,299.592940,299.050538,297.896731,...,302.527189,301.203225,300.461623,299.956024,296.966999,296.987533,296.607472,295.628513,294.564037,293.814723
128,2025-09,DOM,300.267422,0.815065,0.932228,128,299.500889,298.901833,297.842733,296.753960,...,301.049240,300.157069,299.679173,299.946984,296.924344,296.550108,295.661848,294.479999,293.693839,293.554928


In [12]:
clim_covs=pd.merge(inter_data,clim_data,on=["time","region"])
clim_covs
clim_covs= pd.merge(clim_covs,nino,on="time")#Adding el nino index (global index so we cannot join by region)
clim_covs

,time,region,spa01,spe01,spa03,spe03,spa06,spe06,spa12,spe12,...,mx2t24_L4.0,mx2t24_L5.0,mx2t24_L6.0,mn2t24_L1.0,mn2t24_L2.0,mn2t24_L3.0,mn2t24_L4.0,mn2t24_L5.0,mn2t24_L6.0,ssta
0,2015-01,DOM,-0.517534,-0.614625,-0.240362,-0.304545,-0.349382,-0.371714,-1.297219,-1.150566,...,300.992990,301.426667,302.432747,293.653464,293.378766,293.739177,294.505283,295.369165,296.234461,0.477419
1,2015-02,DOM,1.082209,1.113880,-0.182529,-0.233308,-0.269710,-0.304991,-1.026904,-0.938466,...,301.630720,302.482374,303.229442,293.592826,293.830732,294.573811,295.542603,296.380792,296.735205,0.750000
2,2015-03,DOM,1.029790,1.073085,0.773866,0.734489,-0.230838,-0.295462,-1.016961,-0.913970,...,302.500986,303.340244,303.619429,293.968395,294.677446,295.489687,296.385210,296.784302,296.974068,0.438710
3,2015-04,DOM,-0.247234,-0.309503,0.775546,0.760518,-0.151664,-0.130728,-1.126334,-1.031561,...,303.424765,303.731316,303.526471,294.568748,295.494279,296.287564,296.788201,296.941981,296.986188,0.836667
4,2015-05,DOM,-2.520282,-2.028042,-1.072317,-0.983117,-1.057238,-1.101611,-1.391256,-1.239282,...,303.935797,303.611964,302.807706,295.737503,296.548509,297.007120,297.173793,297.105166,296.762237,0.841935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,2025-06,DOM,-0.742095,-0.667191,0.013596,-0.003682,-0.421099,-0.440258,0.187191,0.130215,...,303.444969,302.536345,301.304584,296.373418,296.786608,297.098190,297.038888,296.614203,295.639563,0.056667
126,2025-07,DOM,-0.684034,-0.491420,-0.192809,-0.140606,-0.539482,-0.549485,-0.108972,-0.172499,...,302.500054,301.353363,300.448711,296.639351,297.028190,296.991677,296.623259,295.686722,294.540026,-0.083871
127,2025-08,DOM,-0.715331,-0.661631,-1.000523,-0.949739,-0.833009,-0.869668,-0.278329,-0.350477,...,301.203225,300.461623,299.956024,296.966999,296.987533,296.607472,295.628513,294.564037,293.814723,-0.338710
128,2025-09,DOM,-1.409213,-1.535912,-1.271335,-1.135710,-0.506093,-0.510955,0.056568,0.023770,...,300.157069,299.679173,299.946984,296.924344,296.550108,295.661848,294.479999,293.693839,293.554928,-0.423333


In [13]:
clim_covs.ssta

,ssta
0,0.477419
1,0.750000
2,0.438710
3,0.836667
4,0.841935
...,...
125,0.056667
126,-0.083871
127,-0.338710
128,-0.423333


In [14]:

#Load epidemology data
df = pd.read_csv("%s_epi_training.csv"%name)

df = df[['Month', 'Cases']]

# Convert Month column to datetime
df['Month'] = pd.to_datetime(df['Month'])

# Create time series (Month as time index)
series = TimeSeries.from_dataframe(df, 'Month', 'Cases')

ccvi_ds = xr.open_dataset("%s-ccvi.zs.nc"%name)
ccvi_df = ccvi_ds.to_dataframe().reset_index()
ccvi_df['time'] = pd.to_datetime(ccvi_df['time'])
ccvi_df = ccvi_df.set_index('time')
ccvi_df_monthly = ccvi_df.resample('MS').ffill()
ccvi_df_monthly = ccvi_df_monthly.reset_index()
ccvi_df_monthly['time'] = pd.to_datetime(ccvi_df_monthly['time']).dt.to_period('M')

gdp_ds = xr.open_dataset("%s-gdp_pc.zs.nc"%name)
gdp_df = gdp_ds.to_dataframe().reset_index()
gdp_df['time'] = pd.to_datetime(gdp_df['time'])
gdp_df = gdp_df.set_index('time')
gdp_df_monthly = gdp_df.resample('MS').ffill()
gdp_df_monthly = gdp_df_monthly.reset_index()
gdp_df_monthly['time'] = pd.to_datetime(gdp_df_monthly['time']).dt.to_period('M')

pop_ds = xr.open_dataset("%s-population.zs.nc"%name)
pop_df = pop_ds.to_dataframe().reset_index()
pop_df['time'] = pd.to_datetime(pop_df['time'])
pop_df = pop_df.set_index('time')
pop_df_monthly = pop_df.resample('MS').ffill()
pop_df_monthly = pop_df_monthly.reset_index()
# Convert time to Year-Month (Period[M])
pop_df_monthly['time'] = pd.to_datetime(pop_df_monthly['time']).dt.to_period('M')

In [15]:
pop_df_monthly

,time,region,pop_count,area,pop_density
0,2015-01,DOM,1.024492e+07,48092.054193,213.027326
1,2015-02,DOM,1.024492e+07,48092.054193,213.027326
2,2015-03,DOM,1.024492e+07,48092.054193,213.027326
3,2015-04,DOM,1.024492e+07,48092.054193,213.027326
4,2015-05,DOM,1.024492e+07,48092.054193,213.027326
...,...,...,...,...,...
116,2024-09,DOM,1.123627e+07,48092.054193,233.640953
117,2024-10,DOM,1.123627e+07,48092.054193,233.640953
118,2024-11,DOM,1.123627e+07,48092.054193,233.640953
119,2024-12,DOM,1.123627e+07,48092.054193,233.640953


In [16]:
test=pd.merge(clim_covs,pop_df_monthly,on=["time","region"])
test

,time,region,spa01,spe01,spa03,spe03,spa06,spe06,spa12,spe12,...,mn2t24_L1.0,mn2t24_L2.0,mn2t24_L3.0,mn2t24_L4.0,mn2t24_L5.0,mn2t24_L6.0,ssta,pop_count,area,pop_density
0,2015-01,DOM,-0.517534,-0.614625,-0.240362,-0.304545,-0.349382,-0.371714,-1.297219,-1.150566,...,293.653464,293.378766,293.739177,294.505283,295.369165,296.234461,0.477419,1.024492e+07,48092.054193,213.027326
1,2015-02,DOM,1.082209,1.113880,-0.182529,-0.233308,-0.269710,-0.304991,-1.026904,-0.938466,...,293.592826,293.830732,294.573811,295.542603,296.380792,296.735205,0.750000,1.024492e+07,48092.054193,213.027326
2,2015-03,DOM,1.029790,1.073085,0.773866,0.734489,-0.230838,-0.295462,-1.016961,-0.913970,...,293.968395,294.677446,295.489687,296.385210,296.784302,296.974068,0.438710,1.024492e+07,48092.054193,213.027326
3,2015-04,DOM,-0.247234,-0.309503,0.775546,0.760518,-0.151664,-0.130728,-1.126334,-1.031561,...,294.568748,295.494279,296.287564,296.788201,296.941981,296.986188,0.836667,1.024492e+07,48092.054193,213.027326
4,2015-05,DOM,-2.520282,-2.028042,-1.072317,-0.983117,-1.057238,-1.101611,-1.391256,-1.239282,...,295.737503,296.548509,297.007120,297.173793,297.105166,296.762237,0.841935,1.024492e+07,48092.054193,213.027326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,2024-09,DOM,-0.385924,-0.329014,0.222249,0.311657,0.391587,0.426905,0.067191,0.004262,...,297.300722,297.026674,295.990720,294.743856,293.937170,293.770496,-0.246667,1.123627e+07,48092.054193,233.640953
117,2024-10,DOM,-2.158332,-1.454722,-1.263836,-0.920452,0.230296,0.258241,0.226905,0.139478,...,297.118115,296.093508,294.882175,294.061819,293.873787,294.072614,-0.380645,1.123627e+07,48092.054193,233.640953
118,2024-11,DOM,0.905513,0.990190,-0.255712,-0.288860,0.580443,0.609501,0.592796,0.521181,...,295.690229,294.465971,293.855283,293.586105,293.884623,294.597977,-0.166667,1.123627e+07,48092.054193,233.640953
119,2024-12,DOM,1.360153,1.349144,0.506544,0.500408,0.701150,0.729518,0.755312,0.721063,...,294.783233,293.921222,293.726251,293.889502,294.569336,295.589963,-0.661290,1.123627e+07,48092.054193,233.640953


In [20]:
test.to_csv("Covariates_data_%s"%name)